In [19]:
# Ziwei Qiu, ziweiqiu@g.harvard.edu
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler
import pandas as pd
import sys
from math import log2, floor
import dimod
import numpy as np
import os

In [15]:
def build_knapsack_dqm(costs, weights, weight_capacity, max_num_of_pieces, verbose = False):
    """Construct DQM for the generalized knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        max_num_of_pieces(int):
            Maximum allowable pieces for each item
    Returns:
        Discrete quadratic model instance
    """
    max_num_of_pieces += 1 # also take into account the value 0
    pieces = range(max_num_of_pieces)
    
    # First guess the lagrange
    lagrange = max(costs)*0.2
    if verbose:
        print('lagrange:',lagrange)

    # Number of objects
    x_size = len(costs)

    # Lucas's algorithm introduces additional slack variables to
    # handle the inequality. M+1 binary slack variables are needed to
    # represent the sum using a set of powers of 2.
    M = floor(log2(weight_capacity))
    num_slack_variables = M + 1

    # Slack variable list for Lucas's algorithm. The last variable has
    # a special value because it terminates the sequence.
    y = [2**n for n in range(M)]
    y.append(weight_capacity + 1 - 2**M)
    
    ##@  Discrete Quadratic Model @##
    dqm = DiscreteQuadraticModel()
    
    #@ Add variables @##
    for k in range(x_size):
        dqm.add_variable(max_num_of_pieces, label='x' + str(k))

    for k in range(num_slack_variables):
        dqm.add_variable(2, label='y' + str(k)) # either 0 or 1

    ##@ Hamiltonian xi-xi terms ##
    for k in range(x_size):
        dqm.set_linear('x' + str(k), lagrange * (weights[k]**2) * (np.array(pieces)**2) - costs[k]*pieces)


    # # Hamiltonian xi-xj terms
    for i in range(x_size):
        for j in range(i + 1, x_size):
            biases_dict = {}
            for piece1 in pieces:
                for piece2 in pieces:
                    biases_dict[(piece1, piece2)]=(2 * lagrange * weights[i] * weights[j])*piece1*piece2

            dqm.set_quadratic('x' + str(i), 'x' + str(j), biases_dict)

    # Hamiltonian y-y terms
    for k in range(num_slack_variables):
        dqm.set_linear('y' + str(k), lagrange*np.array([0,1])* (y[k]**2))

    # Hamiltonian yi-yj terms 
    for i in range(num_slack_variables):
        for j in range(i + 1, num_slack_variables): 
            dqm.set_quadratic('y' + str(i), 'y' + str(j), {(1,1):2 * lagrange * y[i] * y[j]})

    # Hamiltonian x-y terms
    for i in range(x_size):
        for j in range(num_slack_variables):
            biases_dict = {}
            for piece1 in pieces:
                biases_dict[(piece1, 1)]=-2 * lagrange * weights[i] * y[j]*piece1

            dqm.set_quadratic('x' + str(i), 'y' + str(j), biases_dict) 
    
    return dqm

def solve_knapsack(costs, weights, weight_capacity, max_num_of_pieces, sampler = None, verbose = False):
    """Construct DQM and solve the generalized knapsack problem
    Args:
        costs (array-like):
            Array of costs associated with the items
        weights (array-like):
            Array of weights associated with the items
        weight_capacity (int):
            Maximum allowable weight
        max_num_of_pieces(int):
            Maximum allowable pieces for each item
        sampler (DQM sampler instance or None):
            A DQM sampler instance or None, in which case
            LeapHybridSampler is used by default        
    Returns:
        Tuple:
            List of indices of selected items
            List of number of pieces for each selected item
            Solution energy
    """
    
    dqm = build_knapsack_dqm(costs, weights, weight_capacity, max_num_of_pieces)
    
    if sampler is None:
        sampler = LeapHybridDQMSampler()
    sampleset = sampler.sample_dqm(dqm)
    sample = sampleset.first.sample
    energy = sampleset.first.energy

    if verbose:
        print("Solution: ", sample)
        print("Solution energy: ", energy)

    selected_item_indices = []
    selected_item_pieces = []
    for varname, value in sample.items():
        # For each "x" variable, check whether its value is set, which
        # indicates that the corresponding item is included in the
        # knapsack
        if value and varname.startswith('x'):
            # The index into the weight array is retrieved from the
            # variable name
            selected_item_indices.append(int(varname[1:]))
            selected_item_pieces.append(value)
    return selected_item_indices, selected_item_pieces, energy

In [16]:
# Load Data
data_file_name = os.path.join(os.getcwd(),'data/large.csv')
df = pd.read_csv(data_file_name, names=['cost', 'weight'])
df = df.head(10)
df

,cost,weight
0,77,95
1,44,70
2,15,85
3,67,31
4,75,100
5,91,1
6,2,7
7,46,53
8,16,35
9,68,11


In [18]:
# parse input data
costs = df['cost']
weights = df['weight']
weight_capacity = 80
max_num_of_pieces = 5

selected_item_indices, selected_item_pieces, energy = solve_knapsack(costs, weights, weight_capacity, max_num_of_pieces, verbose=True)

selected_weights_each = list(df.loc[selected_item_indices,'weight'])
selected_weights = np.multiply(np.array(selected_item_pieces), np.array(selected_weights_each))
selected_costs_each = list(df.loc[selected_item_indices,'cost'])
selected_costs = np.multiply(np.array(selected_item_pieces), np.array(selected_costs_each))
print("Selected item numbers:", selected_item_indices)
print("Number of pieces:", selected_item_pieces)
print("Selected item weights: {}, total = {}".format(selected_weights, sum(selected_weights)))
print("Selected item costs: {}, total = {}".format(selected_costs, sum(selected_costs)))

Solution:  {'x0': 0, 'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0, 'x5': 5, 'x6': 0, 'x7': 0, 'x8': 0, 'x9': 4, 'y0': 1, 'y1': 0, 'y2': 0, 'y3': 0, 'y4': 1, 'y5': 1, 'y6': 0}
Solution energy:  -727.0000000000073
Selected item numbers: [5, 9]
Number of pieces: [5, 4]
Selected item weights: [ 5 44], total = 49
Selected item costs: [455 272], total = 727
